# Basic mBuild tutorial

The first of the MoSDeF tools we will explore is the [mBuild package](http://mosdef-hub.github.io/mbuild/), which utilizes a hierarchical, component-based approach to molecule construction, allowing complex systems to be built using a subset of re-usable parts, just like Legos! In this tutorial we will explore some of mBuild's basic functionality by constructing a linear alkane chain. We'll then examine how mBuild's component-based design approach allows components to be easily swapped to facilitate structural screening.

### Hierarchical design approach

mBuild uses a [composite design pattern](https://en.wikipedia.org/wiki/Composite_pattern) to approach the creation of complex molecular systems. This yields the following features:
* Molecules feature a tree-like hierarchy (as shown in the figure below)
* All components in the hierarchy feature a common data structure (an mBuild `Compound`)
* The lowest level of the hierarchy (the 'leaves') are referred to as particles and are typically individual atoms
* Atomic positions are maintained only at the particle level; higher level components can compute properties based on contained particles

Below is an example of an mBuild molecule hierarchy for an alkylsilane monolayer attached to a crystalline silica surface.
<img src="../graphics/hierarchical_design_image.png" alt="Drawing" style="width: 700px;"/>

### Primer on using Jupyter notebooks

[Jupyter notebooks](https://jupyter-notebook.readthedocs.io/en/stable/) provide an interactive environment for "developing, documenting, and executing code". Several languages are supported; however here we will be using Python. 

Jupyter notebooks feature two primary types of cells:
1. Markdown cells, like this cell, which contain explanatory text
2. Code cells, that can be executed by either clicking on the "run cell" icon or by hitting SHIFT + ENTER.

Cells do not have to be executed in order (however the cells in this tutorial are designed to be executed sequentially), and the order in which cells have been executed is recorded by the bracketed number to the left of the code cell (e.g. [ 1 ]). When a cell is executed you will first see an asterisk (i.e. [ * ]) which means that the cell is still running. When the asterisk is replaced by a number this means the execution has completed.

### Importing mBuild

To begin using mBuild we need to import the mBuild package, which is available through the [Anaconda](https://anaconda.org/conda-forge/mbuild) package managers. mBuild can also be downloaded from source, which is hosted on [Github](https://github.com/mosdef-hub/mbuild).

Here, we'll import the mBuild package along with a visualize routine that will allow us to view our molecules along the way. The `%matplotlib notebook` routine is a Jupyter 'magic' command that allows us to interactively view matplotlib figures within a notebook.

In [ ]:
%matplotlib notebook
import mbuild as mb

### The `Compound` class

The base class of mBuild is the `Compound` class, which defines the primary building block used for constructing molecules. Molecules are constructed hierarchically; however, each level of the hierarchy inherits from the `Compound` class. This means that `Compounds` may contain other `Compounds`, and that the same methods and attributes are present for molecule components at any level of the hierarchy. There are several ways to interrogate what is contained within a `Compound` (e.g. `type`, `dir`, `list`). mBuild `Compounds` feature [a variety of useful methods and attributes](https://mbuild.mosdef.org/en/stable/topic_guides/data_structures.html#) to facilitate system construction.

#### Exercise
Try using the `type` and `dir` functions to explore the `Compound` data type and attributes.

In [ ]:
my_compound = mb.Compound()
type(my_compound)
dir(my_compound)

### Creating `Compounds`

There are several ways that `Compounds` can be created with mBuild. The simplest is to construct them from the particle level. Standard mBuild protocol is to define particles' names according to their elemental symbol (e.g. `'C'`), or to preface their names by an underscore for coarse-grained beads (e.g. `'_CH4'`). This aids in the atomtyping and forcefield application process (using the Foyer package) which we will get back to later.

Now, lets create a simple carbon `Compound`. [Several arguments are available](https://mbuild.mosdef.org/en/stable/topic_guides/data_structures.html#compound) to set various `Compound` attributes upon instantiation. Here, we'll use the `name` argument to specify the element of our `Compound` and the `pos` argument to specify the location of the `Compound` in Cartesian space.

**Note:** mBuild expects all distance units to be in nanometers.

In [ ]:
carbon = mb.Compound(name='C', pos=[1.0, 2.0, 3.0])
carbon.visualize()

Individual atoms are boring. Let's try now to create a simple CH2 moiety. (Don't worry about the undercoordinated carbon; we'll be using this later to piece together an alkane chain.)

The first step we need to take is to create an empty mBuild `Compound` to add our particles to (we can give this `Compound` a name if we'd like).

In [ ]:
ch2 = mb.Compound()
ch2 = mb.Compound(name='CH2')

Now we need to create three particles: one carbon and two hydrogens. We'll manually set the atomic positions such that they represent realistic atomic spacings.

#### Exercise
Define two hydrogen particles, with variables titled `hydrogen1` and `hydrogen2` at distances of 0.1 nm from the carbon particle.

In [ ]:
carbon = mb.Compound(pos=[0.0, 0.0, 0.0], name='C')
hydrogen1 = mb.Compound(pos=[0.1, 0.0, 0.0], name='H')
hydrogen2 = mb.Compound(pos=[-0.1, 0.0, 0.0], name='H')

As described earlier, the hierarchical design approach used by mBuild allows `Compounds` to contain other `Compounds`. To add our three particles to the hierarchy of our CH2 `Compound` we can use the `add` function. All we need to provide are the variable references to these three particles in a list-like format.

In [ ]:
ch2.add([carbon, hydrogen1, hydrogen2])

We can use the `particles` method to view the particles contained by a `Compound`. This method is written as a generator to conserve memory for large systems, so we'll need to convert to a `list`.

#### Exercise

Generate a list of the particles within the CH2 `Compound`.

In [ ]:
list(ch2.particles())

As we can see, our carbon particle and two hydrogen particles are now contained within our CH2 `Compound`. Now let's visualize our `Compound` to confirm we built this correctly.

In [ ]:
ch2.visualize(backend="nglview")

Looking good! However, although we've added our three particles to the CH2 `Compound`, we have yet to define any bonds between them. To accomplish this, we can use the `Compound.add_bond()` method to specify our two C-H bonds.

In [ ]:
ch2.add_bond((carbon, hydrogen1))
ch2.add_bond((carbon, hydrogen2))
ch2.visualize(backend="nglview")

Visually we now see that our CH2 `Compound` contains three particles and two C-H bonds.

### Reusing components

It would be quite tedious to have to go through each of the above steps every time we wanted to create a new CH2 `Compound`. However, this problem is easily solved by wrapping these routines together into a class.

Here, we'll create a class for our CH2 moiety using the same approach we just took above so that we can easily reuse this piece when constructing more complex molecules.

In [ ]:
%matplotlib notebook
import mbuild as mb

class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        
        carbon = mb.Compound(pos=[0.0, 0.0, 0.0], name='C')
        hydrogen1 = mb.Compound(pos=[0.1, 0.0, 0.0], name='H')
        hydrogen2 = mb.Compound(pos=[-0.1, 0.0, 0.0], name='H')
        self.add([carbon, hydrogen1, hydrogen2])
        self.add_bond((carbon, hydrogen1))
        self.add_bond((carbon, hydrogen2))

As we can see, our class definition contains the same commands we just used to the create the CH2 `Compound` above; however, we have replaced `ch2` with `self` so that these commands will be performed on any instance of our `CH2` class. Additionally, since we want our class instance to be an mBuild `Compound`, we specify that our `CH2` class should inherit from `mb.Compound`.

#### Exercise
Create an instance of the CH2 class and visualize the `Compound`.

In [ ]:
ch2 = CH2()
ch2.name = 'myCH2'
ch2.visualize(backend="nglview")

While there are instances where creating `Compounds` particle-by-particle is useful, this process can get a bit tedious. It's much easier to create them by loading in pre-assembled building blocks. These can easily be created using software such as [Avogadro](https://avogadro.cc/). The `load()` function can create mBuild `Compounds` from a variety of common file formats (e.g. PDB, MOL2) that contain particle positions and bonds. Here, we'll create the same CH2 `Compound` by loading from a PDB file.

**Note:** mBuild does not infer bonds. They must be explicitly defined in your code or in an input structure file.

In [ ]:
ch2 = mb.load('ch2.pdb')
ch2.visualize(backend="nglview")

### Connecting components

We've already found that particles can be connected (i.e. bonded) by using the `add_bond` routine; however, this does not actually move the atoms in space, and it would become burdensome to need to manually update the position of each atom. This is where [mBuild's `Port` class](http://mosdef-hub.github.io/mbuild/data_structures.html#mbuild.port.Port) comes into play. `Ports` in the most general sense define a location in space; however, in most cases these can be thought of as dangling bonds.

Let's test this functionality by using `Ports` instead of `add_bond` to create CH2. First, we'll create an empty `Compound` for CH2 that we will add three particles to at unrealistic locations.

In [ ]:
ch2 = mb.Compound()
carbon = mb.Compound(pos=[0.0, 0.0, 0.0], name='C')
hydrogen1 = mb.Compound(pos=[0.5, 0.0, 0.0], name='H')
hydrogen2 = mb.Compound(pos=[1, 0.0, 0.0], name='H')
ch2.add([carbon, hydrogen1, hydrogen2])
ch2.visualize(backend="nglview")

Now we'll instantiate the `Port` class. We can attach the `Port` to the carbon atom by using the `anchor` attribute. This allows mBuild to know which atoms to create bonds between when two `Ports` are connected. We can also provide an `orientation` vector to give our `Port` a desired direction, and can use the `separation` argument to shift our `Port` from the position of the anchor particle. Since we're going to be connecting to a hydrogen, we will shift our `Port` roughly half of a C-H bond length.

In [ ]:
port_C = mb.Port(anchor=carbon, orientation=[1, 0, 0], separation=0.05)
type(port_C)

We now need to add this `Port` to the containment hierarchy of our CH2 molecule, again using the `add` method. We can also provide a descriptive label for our `Port` that we can use for easy access.

In [ ]:
ch2.add(port_C, label='right')
ch2['right']

Now we need to add another `Port` to the carbon particle and one `Port` to each hydrogen particle, giving each of these distinct labels. We'll first add another `Port` to carbon and a `Port` on one of the hydrogens.

In [ ]:
port2_C = mb.Port(anchor=carbon, orientation=[-1, 0, 0], separation=0.05)
ch2.add(port2_C, label='left')

port1_H = mb.Port(anchor=hydrogen1, orientation=[1, 0, 0], separation=0.05)
ch2.add(port1_H, label='H1')

ch2.visualize(show_ports=True, backend="nglview")

#### Exercise
Create a `Port` on the second hydrogen atom and add to the CH2 `Compound` with the label `'H2'`.

In [ ]:
port2_H = mb.Port(anchor=hydrogen2, orientation=[1, 0, 0], separation=0.05)
ch2.add(port2_H, label='H2')

visualize(ch2)

The `force_overlap` function can be used to force the overlap of two `Ports` by performing a coordinate transform on one of the two `Compounds` that should be connected. This will also create a bond between the anchor particles of each `Port`. We'll use this function here to connect one hydrogen to the carbon particle.

In [ ]:
mb.force_overlap(move_this=hydrogen1,
                 from_positions=ch2['H1'],
                 to_positions=ch2['right'])
visualize(ch2)

#### Exercise
Connect the second hydrogen to carbon to complete the CH2 molecule (i.e. connect `Port` `'H2'` to `Port` `'left'`) and visualize.

In [ ]:
mb.force_overlap(move_this=hydrogen2,
                 from_positions=ch2['H2'],
                 to_positions=ch2['left'])
visualize(ch2)

### Building larger `Compounds`

In general, connecting atoms together using `Ports` to create small moieties is unnecessary as these can be loaded from structure files; however, this functionality is important for creating larger and more complex `Compounds`.

#### Exercise
Define a `Port` (labeled `'down'`) on the carbon particle so that we can connect CH2 `Compounds` together.

In [ ]:
%matplotlib notebook
from visualize import visualize
import mbuild as mb

class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        up_port = mb.Port(anchor=carbon, orientation=[0, 0, 1], separation=0.075)
        down_port = mb.Port(anchor=carbon, orientation=[0, 0, -1], separation=0.075)
        self.add(up_port, label='up')
        self.add(down_port, label='down')

Now we'll explore how `Ports` can be used to connect `Compounds` by connecting two CH2 groups to create a C2H4 group. We'll first use mBuild's `clone` function to create two deep copies of our CH2 `Compound`.

We can use the `translate` function to move one copy so that they are not on top of one another. We'll also add these to a temporary `Compound` so that we can visualize both simultaneously.

In [ ]:
ch2 = CH2()
ch2_copy1 = mb.clone(ch2)
ch2_copy2 = mb.clone(ch2)
ch2_copy2.translate([1, 1, 1])
temp_compound = mb.Compound()
temp_compound.add((ch2_copy1, ch2_copy2))
visualize(temp_compound)

#### Exercise
Create a C2H4 `Compound` by using `force_overlap` to connect the `'up'` `Port` of `ch2_copy1` with the `'down'` `Port` of `ch2_copy2`.

In [ ]:
ch2_copy1 = mb.clone(ch2)
ch2_copy2 = mb.clone(ch2)
ch2_copy2.translate([1, 1, 1])

mb.force_overlap(move_this=ch2_copy1,
                 from_positions=ch2_copy1['up'],
                 to_positions=ch2_copy2['down'])

We'll now create an empty `Compound` for our C2H4 molecule and add the two CH2 copies that are now connected.

In [ ]:
c2h4 = mb.Compound()
c2h4.add((ch2_copy1, ch2_copy2))
visualize(c2h4)

### Building a linear alkane

Now that we've explored the basics of creating mBuild `Compounds` and connecting them together, we'll use this approach to create a slightly more complex molecule, a linear butane.

First, we'll update our CH2 class definition to add two ports, oriented in +z and -z.

In [ ]:
%matplotlib notebook
from visualize import visualize
import mbuild as mb

class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        up_port = mb.Port(anchor=carbon, orientation=[0, 0, 1], separation=0.075)
        down_port = mb.Port(anchor=carbon, orientation=[0, 0, -1], separation=0.075)
        self.add(up_port, label='up')
        self.add(down_port, label='down')

We could approach our butane construction by connecting two CH2 moieties and two CH3 moieties. Alternatively, we could connect four CH2 moieties and cap the ends of the chain with hydrogen atoms. We'll go ahead and take the latter approach. As such, we'll need to also define a class for a hydrogen atom featuring a single port.

In [ ]:
class Hydrogen(mb.Compound):
    def __init__(self):
        super(Hydrogen, self).__init__()
        
        self.add(mb.Compound(name='H'))
        up_port = mb.Port(anchor=self[0], orientation=[0, 0, 1], separation=0.05)
        self.add(up_port, 'up')

We now have all of the pieces necessary to create a butane molecule. To begin, we'll instantiate an empty mBuild `Compound` to add our pieces to.

In [ ]:
butane = mb.Compound()

Now, we'll create one of our CH3 ends by connecting a CH2 group and a hydrogen atom. We'll then add these two `Compounds` to our butane, giving them each a label. Note that by providing `ch2[$]` as the label for our CH2 group, mBuild will create a list that any subsequent parts added to the `Compound` with the same label prefix will be appended to.

In [ ]:
hydrogen = Hydrogen()
last_unit = CH2()
mb.force_overlap(move_this=hydrogen,
                 from_positions=hydrogen['up'],
                 to_positions=last_unit['up'])
butane.add(last_unit, label='ch2[$]')
butane.add(hydrogen, label='up-cap')
visualize(butane)

To continue to create our butane molecule, we'll next attach three CH2 groups to the CH3 cap we've just created. This can be set up in a loop, where we'll use `force_overlap` to iteratively attach each new CH2 instantiation to the last unit on the chain.

In [ ]:
for _ in range(3):
    current_unit = CH2()
    mb.force_overlap(move_this=current_unit,
                     from_positions=current_unit['up'],
                     to_positions=last_unit['down'])
    butane.add(current_unit, label='ch2[$]')
    last_unit=current_unit

visualize(butane)

Finally, we need to cap the end of our molecule with a hydrogen atom to complete the creation of butane.

In [ ]:
hydrogen2 = Hydrogen()
mb.force_overlap(move_this=hydrogen2,
                 from_positions=hydrogen2['up'],
                 to_positions=last_unit['down'])
butane.add(hydrogen2, label='down-cap')
visualize(butane)

As shown previously, we can also wrap all of these commands into a class.

In [ ]:
class Butane(mb.Compound):
    def __init__(self):
        super(Butane, self).__init__()
        
        hydrogen = Hydrogen()
        last_unit = CH2()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='ch2[$]')
        self.add(hydrogen, label='up-cap')
        for _ in range(3):
            current_unit = CH2()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='ch2[$]')
            last_unit=current_unit
        hydrogen = Hydrogen()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['down'])
        self.add(hydrogen, label='down-cap')

In [ ]:
butane = Butane()
visualize(butane)

### Creating flexible classes

If we had to create a new class for each molecule we wanted to examine this would still be quite cumbersome if we wanted to screen over a large structural parameter space. However, since each `Compound` is defined as a Python class, one simply needs to define one or more top-level variables as arguments so that a single class definition could be used to create a whole family of molecules. We'll demonstrate that here by modifying the Butane class we've just defined to allow the creation of any linear alkane by adding a `chain_length` argument.

In [ ]:
class Alkane(mb.Compound):
    def __init__(self, chain_length):
        super(Alkane, self).__init__()
        
        hydrogen = Hydrogen()
        last_unit = CH2()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='ch2[$]')
        self.add(hydrogen, label='up-cap')
        for _ in range(chain_length - 1):
            current_unit = CH2()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='ch2[$]')
            last_unit=current_unit
        hydrogen = Hydrogen()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['down'])
        self.add(hydrogen, label='down-cap')

We can now create any linear alkane by simply providing a different value for `chain_length` upon instantiation.

In [ ]:
ethane = Alkane(chain_length=2)
ethane.visualize()

In [ ]:
hexane = Alkane(chain_length=6)
visualize(hexane)

And those are the basics of mBuild! By defining flexible `Compound` classes with several top-level variables, a pool of `Compounds` spanning a large structural parameter space can be created by simply nesting several `for` loops.

In [ ]:
alkanes = mb.Compound()
for chain_length in range(2, 11, 2):
    alkane = Alkane(chain_length=chain_length)
    alkane.translate([len(alkanes.children) / 2, 0, 0])
    alkanes.add(alkane)
visualize(alkanes)

### Coarse Grained Molecule

 Will be used in the example workflow

### Packing functions

Will be used in the example workflow